# Metaklasy

In [33]:
class MyClass(object, metaclass=type):
    pass

In [34]:
mc = MyClass()

In [35]:
type(mc)

__main__.MyClass

In [36]:
type(MyClass)

type

## Statycznie utworzona klasa

In [6]:
class Shape:
    def draw(self):
        pass

class Rectangle(Shape):
    _id = "RECTANGLE"

    def __init__(self, width, height):
        self.width = width
        self.height = height        

    def draw(self):
        print(f'Drawing {Rectangle._id}({self.width}, {self.height})')

In [9]:
rect = Rectangle(20, 40)
rect.draw()

Drawing RECTANGLE(20, 40)


In [10]:
type(Rectangle)

type

In [11]:
Rectangle.__name__

'Rectangle'

## Dynamicznie utworzona klasa - type

In [18]:
def rect_init(self, width, height):
    self.width = width
    self.height = height  

TRectangle = type('TRectangle', (Shape, ), {
    '_id': 'T_RECTANGLE',
    '__init__': rect_init,
    'draw' : lambda self : print(f'Drawing {Rectangle._id}({self.width}, {self.height})'),
    '__doc__': """This is dynamically created type"""
})

In [19]:
type(TRectangle)

type

In [20]:
TRectangle.__name__

'TRectangle'

In [21]:
TRectangle.__dict__

mappingproxy({'_id': 'T_RECTANGLE',
              '__init__': <function __main__.rect_init(self, width, height)>,
              'draw': <function __main__.<lambda>(self)>,
              '__doc__': 'This is dynamically created type',
              '__module__': '__main__'})

In [22]:
TRectangle.__mro__

(__main__.TRectangle, __main__.Shape, object)

In [23]:
rt = TRectangle(100, 200)

In [24]:
rt.draw()

Drawing RECTANGLE(100, 200)


In [27]:
class ColorRectangle(TRectangle):
    def __init__(self, w, h, color):
        super().__init__(w, h)
        self.color = color

In [28]:
color_rect = ColorRectangle(10, 200, "red")

In [30]:
color_rect.color

'red'

In [32]:
color_rect.width

10

# Metaklasa

In [37]:
class MyClass(object, metaclass=type):
    foo = 'bar'

In [38]:
MyClass.foo

'bar'

## Funkcja jako metaklasa

In [50]:
def attributes_to_upper(cls, parents, dict_attrs):
    if "Id" in dict_attrs.keys():
        raise AttributeError("Id attribute is not allowed")
    
    _attrs = ((name.upper(), value)
                for name, value in dict_attrs.items())
    
    attrs_upper = dict(_attrs)
    return type(cls, parents, attrs_upper)

In [52]:
class MyUpperClass(metaclass=attributes_to_upper):
    foo = 'bar'
    Name = 'Classy'
    iD = 42

In [53]:
MyUpperClass.FOO

'bar'

In [54]:
MyUpperClass.NAME

'Classy'

In [55]:
MyUpperClass.ID

42

In [56]:
MyUpperClass.__dict__

mappingproxy({'__MODULE__': '__main__',
              '__QUALNAME__': 'MyUpperClass',
              'FOO': 'bar',
              'NAME': 'Classy',
              'ID': 42,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyUpperClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyUpperClass' objects>,
              '__doc__': None})

In [61]:
class NoIdInside(MyUpperClass, metaclass=attributes_to_upper):
    Id = 665

# Klasy jako metaklasy

In [ ]:
class Metaclass(type):
    @classmethod 
    def __prepare__(mcs, name, bases, **kwargs): 
        return super().__prepare__(name, bases, **kwargs)

    def __new__(mcs, name, bases, class_dict):
        return super().__new__(mcs, name, bases, class_dict)  
    
    def __init__(cls, name, bases, class_dict, **kwargs): 
        super().__init__(name, bases, class_dict) 

    def __call__(cls, *args, **kwargs): 
        return super().__call__(*args, **kwargs)

In [62]:
class RevealingMeta(type): 
    """Metaclass - reports steps of class creation"""
    
    def __new__(mcs, name, bases, namespace, **kwargs): 
        print(mcs, "METACLASS __new__ called") 
        return super().__new__(mcs, name, bases, namespace) 
 
    @classmethod 
    def __prepare__(mcs, name, bases, **kwargs): 
        print(mcs, " METACLASS __prepare__ called") 
        return super().__prepare__(mcs, name, bases, **kwargs) 
 
    def __init__(cls, name, bases, namespace, **kwargs): 
        print(cls, " METACLASS __init__ called") 
        super().__init__(name, bases, namespace) 
 
    def __call__(cls, *args, **kwargs): 
        print(cls, " METACLASS __call__ called") 
        return super().__call__(*args, **kwargs) 

In [63]:
class RevealingClass(metaclass=RevealingMeta):
    def __new__(cls):
        print(cls, "__new__ called")
        return super().__new__(cls)
    
    def __init__(self):
        print(self, "__init__ called")
        super().__init__()

<class '__main__.RevealingMeta'>  METACLASS __prepare__ called
<class '__main__.RevealingMeta'> METACLASS __new__ called
<class '__main__.RevealingClass'>  METACLASS __init__ called


In [64]:
instance_rc = RevealingClass()

<class '__main__.RevealingClass'>  METACLASS __call__ called
<class '__main__.RevealingClass'> __new__ called
<__main__.RevealingClass object at 0x000002A8A9456200> __init__ called


## Przykłady metaklasy

In [65]:
from typing import Any, Mapping
import inflection

class CaseInterpolationDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        super().__setitem__(inflection.underscore(key), value)


In [66]:
lookup_numbers = CaseInterpolationDict()

In [68]:
lookup_numbers['one'] = 1
lookup_numbers['TwentyOne'] = 21

In [69]:
lookup_numbers

{'one': 1, 'TwentyOne': 21, 'twenty_one': 21}

In [70]:
class CaseInterpolatedAttributes(type):
    """Metaclass for names interpolation"""

    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        return CaseInterpolationDict()

In [71]:
class InterpolatedNames(metaclass=CaseInterpolatedAttributes):
    pass

class User(InterpolatedNames):
    IdUser = 42

    def __init__(self, firstName: str, lastName: str):
        self.firstName = firstName
        self.lastName = lastName

    def getDisplayName(self):
        return f"{self.firstName} {self.lastName}"
    
    def greetUser(self):
        return f"Hello {self.getDisplayName()}!"


In [72]:
User.id_user

42

In [73]:
User.IdUser

42

In [74]:
admin = User("Jan", "Kowalski")

In [75]:
admin.get_display_name()

'Jan Kowalski'

# Jak działa namedtuple?

In [76]:
from collections import namedtuple

Point = namedtuple('Point', 'x, y')

In [77]:
Point.__dict__

mappingproxy({'__doc__': 'Point(x, y)',
              '__slots__': (),
              '_fields': ('x', 'y'),
              '_field_defaults': {},
              '__new__': <staticmethod(<function Point.__new__ at 0x000002A8AAEA76D0>)>,
              '_make': <classmethod(<function Point._make at 0x000002A8AAEA7760>)>,
              '_replace': <function collections.Point._replace(self, /, **kwds)>,
              '__repr__': <function collections.Point.__repr__(self)>,
              '_asdict': <function collections.Point._asdict(self)>,
              '__getnewargs__': <function collections.Point.__getnewargs__(self)>,
              '__match_args__': ('x', 'y'),
              'x': _tuplegetter(0, 'Alias for field number 0'),
              'y': _tuplegetter(1, 'Alias for field number 1'),
              '__module__': '__main__'})

In [78]:
pt = Point(10, 20)

In [79]:
pt

Point(x=10, y=20)

In [80]:
from typing import Iterable, Iterator, Union


FieldNames = Union[str, Iterable[str]]

In [84]:
def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()

    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')

    return tuple(names)

In [85]:
parse_identifiers('x, y')

('x', 'y')

In [86]:
def struct_factory(cls_name: str, field_names: str):
    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)

        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join(f"{name}={value!r}" for name,
                           value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f"{cls_name}({values})"

    cls_attrs = dict(
        __slots__ = slots,
        __init__ = __init__,
        __repr__ = __repr__,
        __iter__ = __iter__
    )

    return type(cls_name, (object, ), cls_attrs)

In [87]:
Point = struct_factory('Point', 'x, y')

In [88]:
pt = Point(10, 20)

In [89]:
pt.x

10

In [90]:
pt.y

20

In [91]:
pt

Point(x=10, y=20)

In [92]:
for i in pt:
    print(i)

10
20


In [93]:
pt.x = 30

In [94]:
pt.x

30

# Jak działa NamedTuple?

In [95]:
from typing import NamedTuple

In [96]:
class Employee(NamedTuple):
    name: str
    id: int

In [97]:
Employee.__dict__

mappingproxy({'__doc__': 'Employee(name, id)',
              '__slots__': (),
              '_fields': ('name', 'id'),
              '_field_defaults': {},
              '__new__': <staticmethod(<function Employee.__new__ at 0x000002A8AAEA7B50>)>,
              '_make': <classmethod(<function Employee._make at 0x000002A8AAEA5C60>)>,
              '_replace': <function collections.Employee._replace(self, /, **kwds)>,
              '__repr__': <function collections.Employee.__repr__(self)>,
              '_asdict': <function collections.Employee._asdict(self)>,
              '__getnewargs__': <function collections.Employee.__getnewargs__(self)>,
              '__match_args__': ('name', 'id'),
              'name': _tuplegetter(0, 'Alias for field number 0'),
              'id': _tuplegetter(1, 'Alias for field number 1'),
              '__module__': '__main__',
              '__annotations__': {'name': str, 'id': int},
              '__orig_bases__': (<function typing.NamedTuple(typename

In [98]:
emp = Employee("Jan", 665)

In [99]:
emp

Employee(name='Jan', id=665)

In [100]:
class MetaRecord(type):
    """Metaclass for all records"""
    
    def __new__(mcl, classname, bases, classdict):
        
        # Define as local functions __init__ & __repr__ that we'll use 
        # in the new class
        def __init__(self, **kwargs):
            """__init__ is simple: first, set attributes without
               explicit values to their defaults; then, set those
               explicitly passed in kw.
            """
            for k in self.__dflts__:
                if not k in kwargs:
                    setattr(self, k, self.__dflts__[k])
            for k in kwargs:
                setattr(self, k, kwargs[k])

        def __repr__(self):
            """__repr__ is minimal: shows only attributes that
               differ from default values, for compactness.
            """
            rep = [f'{k}={getattr(self, k)!r}' for k in self.__dflts__ if getattr(self, k) != self.__dflts__[k]]
            return f"{classname}({', '.join(rep)})"
        
        newdict = { 
            '__slots__': [], 
            '__dflts__': {},
            '__init__' : __init__, 
            '__repr__' : __repr__,
        }
        
        for k in classdict:
            if k.startswith('__') and k.endswith('__'):
                if k in newdict:
                    warnings.warn(f"Cannot set attr {k!r} in bunch-class {classname!r}")
                else:
                    newdict[k] = classdict[k]
            else:
                newdict['__slots__'].append(k)
                newdict['__dflts__'][k] = classdict[k]

        return super().__new__(mcl, classname, bases, newdict)


class Record(metaclass=MetaRecord):
    pass

In [113]:
class Pixel(Record):
    x = 0
    y = 0
    color = 'red'

In [102]:
Pixel.__dict__

mappingproxy({'__slots__': ['x', 'y', 'color'],
              '__dflts__': {'x': 0, 'y': 0, 'color': 'red'},
              '__init__': <function __main__.MetaRecord.__new__.<locals>.__init__(self, **kwargs)>,
              '__repr__': <function __main__.MetaRecord.__new__.<locals>.__repr__(self)>,
              '__module__': '__main__',
              'color': <member 'color' of 'Pixel' objects>,
              'x': <member 'x' of 'Pixel' objects>,
              'y': <member 'y' of 'Pixel' objects>,
              '__doc__': None})

In [108]:
px = Pixel(x=10, y=20, color="blue")

In [109]:
px.color

'blue'

In [111]:
px

Pixel(x=10, y=20, color='blue')

# ABC i metaklasy

In [121]:
def abstractmethod(function):
    function.__abstract__ = True
    return function 

In [124]:
import functools


class AbstractMeta(type):
    def __new__(metaclass, name, bases, namespace):
        # Create the class instance
        print(f"__new__({metaclass}, {name}, {bases!r}, {namespace!r})")
        cls = super().__new__(metaclass, name, bases, namespace)

        # Collect all local method marked as abstract
        abstracts = set()
        for k, v in namespace.items():
            if getattr(v, '__abstract__', False):
                abstracts.add(k)

        # Look for abstract methods in the base classes and add them to the list of abstracts
        for base in bases:
            for k in getattr(base, '__abstracts__', ()):
                v = getattr(cls, k, None)
                if getattr(v, '__abstract__', False):
                    abstracts.add(k)

        # Store the abstracts in a frozenset so the cannot be modified
        cls.__abstracts__ = frozenset(abstracts)

        # Decorate the __new__ function to check if all abstract functions were implemented
        original_new = cls.__new__
        @functools.wraps(original_new)
        def new(self, *args, **kwargs):
            for k in self.__abstracts__:
                v = getattr(self, k)
                if getattr(v, '__abstract__', False):
                    raise RuntimeError(f'{k} is not implemented')
            return original_new(self, *args, **kwargs)
        
        cls.__new__ = new
        return cls
    
class MyAbc(metaclass=AbstractMeta):
    pass

__new__(<class '__main__.AbstractMeta'>, MyAbc, (), {'__module__': '__main__', '__qualname__': 'MyAbc'})


In [129]:
class Shape(MyAbc):

    @abstractmethod
    def draw(self) -> None:
        pass

class RotableShape(Shape):
    @abstractmethod
    def rotate(self, angle):
        pass

    def draw(self):
        print("Drawing shape")

class Rectangle(RotableShape):
    pass
    # def rotate(self, angle):
    #     print("Rotate rectangle")

r = Rectangle()

__new__(<class '__main__.AbstractMeta'>, Shape, (<class '__main__.MyAbc'>,), {'__module__': '__main__', '__qualname__': 'Shape', 'draw': <function Shape.draw at 0x000002A8AB003490>})
__new__(<class '__main__.AbstractMeta'>, RotableShape, (<class '__main__.Shape'>,), {'__module__': '__main__', '__qualname__': 'RotableShape', 'rotate': <function RotableShape.rotate at 0x000002A8AB0D45E0>, 'draw': <function RotableShape.draw at 0x000002A8AB0D40D0>})
__new__(<class '__main__.AbstractMeta'>, Rectangle, (<class '__main__.RotableShape'>,), {'__module__': '__main__', '__qualname__': 'Rectangle', 'rotate': <function Rectangle.rotate at 0x000002A8AB0D4430>})
